In [1]:
import pandas as pd
import numpy as np
import sys
import librosa
from IPython.display import Audio
import keras
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from pydub import AudioSegment
import os

2024-03-24 15:59:56.791436: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 15:59:57.942977: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def extract_features(y, sr):
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    return {
        'chroma_stft_mean': np.mean(chroma_stft),
        'chroma_stft_std': np.std(chroma_stft),
        'spectral_centroid_mean': np.mean(spectral_centroid),
        'spectral_centroid_std': np.std(spectral_centroid),
        'spectral_bandwidth_mean': np.mean(spectral_bandwidth),
        'spectral_bandwidth_std': np.std(spectral_bandwidth),
        'rolloff_mean': np.mean(rolloff),
        'rolloff_std': np.std(rolloff),
        'zero_crossing_rate_mean': np.mean(zero_crossing_rate),
        'zero_crossing_rate_std': np.std(zero_crossing_rate),
        **{f'mfcc{i}_mean': np.mean(mfcc[i]) for i in range(mfcc.shape[0])},
        **{f'mfcc{i}_std': np.std(mfcc[i]) for i in range(mfcc.shape[0])}
    }

In [3]:
def convert_aac_to_wav(aac_file, wav_file):
    sound = AudioSegment.from_file(aac_file, format="aac")
    sound.export(wav_file, format="wav")




In [3]:
# convert_aac_to_wav('real_reacording/3.aac', 'real_reacording/3.wav')
Audio("real_reacording/3.wav")

In [4]:
from joblib import dump, load
rf_model = load('finalized_model.sav.joblib')
# rf_model = load('model.joblib')

# import pickle
# with open('rf_model.pkl', 'rb') as f:
#     rf_model = pickle.load(f)

# sound = convert_aac_to_wav('real_reacording/1.aac', 'real_reacording/1.wav')

y,sr = librosa.load('real_reacording/3.aac', sr=48000)
features = extract_features(y, 48000)
df = pd.DataFrame(features, index=[0])

prediction = rf_model.predict(df)[0]
print("Predicted emotion:", prediction)


Predicted emotion: fearful


In [4]:
# import pyaudio

# pa = pyaudio.PyAudio()

# print('\navailable devices:')

# for i in range(pa.get_device_count()):
#     dev = pa.get_device_info_by_index(i)
#     name = dev['name'].encode('utf-8')
#     print(i, name, dev['maxInputChannels'], dev['maxOutputChannels'])

# print('\ndefault input & output device:')
# print(pa.get_default_input_device_info())
# print(pa.get_default_output_device_info())


available devices:
0 b'HDA NVidia: HDMI 0 (hw:0,3)' 0 8
1 b'HDA NVidia: HDMI 1 (hw:0,7)' 0 8
2 b'HDA NVidia: HDMI 2 (hw:0,8)' 0 8
3 b'HDA NVidia: HDMI 3 (hw:0,9)' 0 8
4 b'HD-Audio Generic: ALC287 Analog (hw:1,0)' 2 2
5 b'AB13X USB Audio: - (hw:2,0)' 0 2
6 b'hdmi' 0 8
7 b'pulse' 32 32
8 b'default' 32 32

default input & output device:
{'index': 8, 'structVersion': 2, 'name': 'default', 'hostApi': 0, 'maxInputChannels': 32, 'maxOutputChannels': 32, 'defaultLowInputLatency': 0.008684807256235827, 'defaultLowOutputLatency': 0.008684807256235827, 'defaultHighInputLatency': 0.034807256235827665, 'defaultHighOutputLatency': 0.034807256235827665, 'defaultSampleRate': 44100.0}
{'index': 8, 'structVersion': 2, 'name': 'default', 'hostApi': 0, 'maxInputChannels': 32, 'maxOutputChannels': 32, 'defaultLowInputLatency': 0.008684807256235827, 'defaultLowOutputLatency': 0.008684807256235827, 'defaultHighInputLatency': 0.034807256235827665, 'defaultHighOutputLatency': 0.034807256235827665, 'defaultSam

ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave


In [5]:
import pyaudio

CHUNK_SIZE = 1024  
FORMAT = pyaudio.paInt16
CHANNELS = 1  
RATE = 48000  

p = pyaudio.PyAudio()   
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                input=True, frames_per_buffer=CHUNK_SIZE,)


buffer = []  
BUFFER_LENGTH = 70 


try:
    while True:
        data = stream.read(CHUNK_SIZE)
        buffer.append(data)

        if len(buffer) == BUFFER_LENGTH:
            frames = b''.join(buffer) 
            y = np.frombuffer(frames, dtype=np.int16).astype(np.float32) 

            features = extract_features(y, RATE) 
            df = pd.DataFrame(features, index=[0])

            prediction = rf_model.predict(df)[0]
            print("Predicted emotion:", prediction)

            buffer = []  

except KeyboardInterrupt:
    print("Exiting...")
    stream.stop_stream()
    stream.close()
    p.terminate()

ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave


Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Predicted emotion: fearful
Exiting...


In [6]:
# import gradio as gr
# import numpy as np
# import pyaudio
# from joblib import load

# # Load the pre-trained model
# # rf_model = load("finalized_model.sav.joblib")

# # --- PyAudio Configuration ---
# CHUNK_SIZE = 1024
# FORMAT = pyaudio.paInt16
# CHANNELS = 1
# RATE = 48000

# # Feature extraction function (same as original script)
# def extract_features(y, sr):
#     # ... (same feature extraction logic as before) ...
#     return features

# # Function to process audio stream and predict emotion
# def predict_emotion(audio):
#     # Convert audio data to NumPy array
#     y = np.frombuffer(audio, dtype=np.int16).astype(np.float32)

#     # Extract features
#     features = extract_features(y, RATE)
#     df = pd.DataFrame(features, index=[0])

#     # Make prediction
#     prediction = rf_model.predict(df)[0]
#     return prediction

# # Gradio interface definition
# def build_app():
#     # Define the audio input with live streaming enabled
#     # audio_input = gr.Audio(, streaming=True)

#     # Define the text output to display the predicted emotion
#     text_output = gr.Text()

#     # Create the Gradio interface
#     interface = gr.Interface(
#         fn=predict_emotion,
#         inputs=audio_input,
#         outputs=text_output,
#         title="Real-Time Emotion Detection",
#         description="Speak into your microphone and see the predicted emotion!",
#     )

#     return interface

# # Launch the Gradio app
# interface = build_app()
# interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
